In [4]:
import sys
sys.path.append('C:/Program Files (x86)/ArcGIS/Desktop10.3/bin')
sys.path.append('C:/Program Files (x86)/ArcGIS/Desktop10.3/arcpy')
sys.path.append('C:/Program Files (x86)/ArcGIS/Desktop10.3/ArcToolbox/Scripts')

In [5]:
from subprocess import call
import os
os.environ["GDAL_DATA"] = "C:/OSGeo4W/share/gdal"
os.environ["GDAL_DRIVER_PATH"] = "C:/OSGeo4W/bin/gdalplugins"
os.environ["PROJ_LIB"] = "C:/OSGeo4W/share/proj"
os.environ["PATH"] = "C:/OSGeo4W/bin;" + os.environ["PATH"] + ";c:/OSGeo4W/apps/msys/bins;C:/OSGeo4W/apps/Python27/Sripts"

This section is used to create a new list to store all county name, so I can use the list to search county in the shapefile. Then I create two new list pop[] and poplist[]. pop[] is a temporary list, which can store the population number in each county and the sum of the population of each county will be stored in list poplist[]. 

In [6]:
import arcpy
from arcpy import env
env.workspace = "U:/Python class/ArcPy Lab/saep_bg10"
saep = "U:/Python class/ArcPy Lab/saep_bg10/saep_bg10.shp"

# create new list to save county name
county = []
mycursor = arcpy.da.SearchCursor("WashingtonFIPS.dbf",["CountyName"])
for row in mycursor:
    county.append(row[0])


# create new list to save population by county
pop = [] 
poplist = []
# count population by county
for i in range(0,len(county)):
    mycursor2 = arcpy.da.SearchCursor(saep , ["CountyName","POP2013"])
    for row in mycursor2:
        if row[0] == county[i]:
            pop.append(row[1])
    #print( county[i] + "county population is " + str(sum(pop)))
    poplist.append(sum(pop))
    pop = []


After I stored the population of each county in poplist[], I created a dictionary countypop{} to connect county name and its population in 2013. When I search the county name, countypop can show me the relative population number of that county. Then I sort countypop[] by using sorted function, and use a for loop to print the top 10 populations of counties.

In [7]:
countypop = {} # create a dictionary: connect county and population
for i in range(0,len(county)):
    countypop[county[i]] = poplist[i]

# sort population by county    
import operator
sorted_pop = sorted(countypop.items(), key = operator.itemgetter(1))

print "The 10 largest total populations in 2013 of counties in Washington are:"

# print top 1o county
for i in range(1, 11):
    print  sorted_pop[-i]
    i = i + 1

The 10 largest total populations in 2013 of counties in Washington are:
(u'King', 1981900.000000001)
(u'Pierce', 814499.9999999999)
(u'Snohomish', 730499.9999999999)
(u'Spokane', 479999.9999999997)
(u'Clark', 435500.00000000047)
(u'Thurston', 260100.00000000006)
(u'Kitsap', 254000.00000000003)
(u'Yakima', 247250.0)
(u'Whatcom', 205800.00000000003)
(u'Benton', 183400.0)


This section is used to create new shapefiles for counties and convert the shapefiles to geojson files. I first defined several variables to store the file name and file path, and used them in the FeatureClassToFeatureClass function and call. I was going to create an another loop to convert the shapefiles to geojson files and after I talked to a classmate, I realized it was unnecessary. I can simpliy add the call after the FeatureClassToFeatureClass function. 

In [8]:
# create a new shapefile for each county
# define featureclasstofeatureclass
# convert shapefiles to geojson files
import os.path

for i in range(0, len(county)):
    outFeatureClass = str(county[i])
    inFeature = "U:/Python class/ArcPy Lab/saep_bg10/saep_bg10.shp"
    outLocation = "U:/Python class/ArcPy Lab/shapefile/"
    outLocation2 = "U:/Python class/ArcPy Lab/geojson/"
    countystr = str(county[i])
    #print outFeatureClass, countystr
    expression = '"CountyName" = ' + "'%s'" %countystr
  
    arcpy.FeatureClassToFeatureClass_conversion(inFeature, outLocation, outFeatureClass, expression)
    call(['C:\\OSGeo4W\\bin\\ogr2ogr',
      '-f','GeoJSON','-t_srs','WGS84',
      '-s_srs','EPSG:2927',
      outLocation2 + outFeatureClass +".geojson",
      outLocation + outFeatureClass + ".shp"])
    
 

I spent at least 10 hours on this assignment. At first, I felt it was hard and frustrating, but when it's done, I just realize it is not that hard and feel so happy. I learned a lot of new stuff from the coding process and after talking with a classmate I feel it's amazing to see people think in different ways about solving problems.